Para acessar o servidor do PS1 olhe os sites:

http://ps1images.stsci.edu/ps1_dr2_query.html

http://ps1images.stsci.edu/ps1_dr2_api.html

## Abaixo estão os inputs

NSIDE
para a resolução

$\textbf{constraints}$

$\mathit{use:}$
- True para usar as restrições;
- False, caso contrário.

$\mathit{type:}$
- galaxy: para extrair galáxias
- star: para extrair estrelas
- qualquer outra input não usa esta restrição


$\mathit{band:}$ para selecionar a banda da restrição
- g, r, i, z e y


$\textbf{params\_flags}$

- $\mathit{use:}$ True para usar as restrições de flags;
- $\mathit{table:}$ usar a 1 = DetectionFlags, 2 = DetectionFlags2 ou 3 = DetectionFlags3.
- $\mathit{band:}$ se refere ao uso da table numa dada banda (g, r, i, z ou y). Isto é importante para is dados que vem classificados em: ginfoFlags, ginfoFlags2, ginfoFlags3, iinfoFlags, iinfoFlags2, iinfoFlags3, rinfoFlags, rinfoFlags2, rinfoFlags3,...

$\textbf{params\_strips}$

- $\mathit{dec\ strips:}$ True: se for restringir a coleta (em declination), False caso contrário.
- $\mathit{dec\ center:}$ O centro (em graus) da strip
- $\mathit{dec\ width :}$ comprimento da strip 
 
$\textbf{hexa\_query}$
- o hexadecimal que contêm os flags a serem ativados de uma das três tabelas.

$\textbf{restart:}$
- Caso False, ele não iniciará do primeiro arquivo, mas sim do último arquivo existente.
- OBS: Caso mude a strip a ser utilizada é preciso deixar True, de forma ao código limpar o diretório e usar somente, para o dado NSIDE, os arquivos desejados.

In [18]:
NSIDE         = 4*256
constraints   = {"use":False, "type":"galaxy", "band": "r"} 
params_flags  = {"use":True, "table":1, "band":"r"}
params_strips = {'dec strips':True,'dec center':-15.,'dec width': 15}
hexa_query    = 0x1003bc88
restart       = False

## Importar bibliotecas --- Aqui começa o código

In [19]:
%matplotlib inline
import mastcasjobs
import casjobs
import healpy as hp
import numpy as np
import requests
import requests.exceptions
import matplotlib.pyplot as plt
from math import *
from astropy.io import ascii, fits
from astropy.table import Table, join, hstack, vstack
from time import time,strftime, gmtime

import os, sys, re
import pylab
import json

#import flags

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib

In [20]:
#Dê como string o usuário e a senha
user      = "1112277644"
pwd       = "Feymann@1"

In [21]:
import getpass
if not os.environ.get('CASJOBS_WSID'):
    os.environ['CASJOBS_WSID'] = user
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = pwd

# Verificar qual o NSIDE mínimo para a conexão de internet

In [22]:
def maxnside():
    theta,phi = 0.,0.#degree
    nsides = [2**x for x in range(12)][1:]
    nsides = np.flip(nsides)
    for nside in nsides:
        pix = hp.ang2pix(nside,theta = theta,phi = phi, lonlat=True) 
        ang, radius = parameters(nside,pix)
        query = query_string(theta,phi,radius)
        jobs  = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
    
        try:
            jobs.quick(query, task_name="python cone search")
            pass
        except Exception:
            return nside

## Estabelecer conexão com o PS1

As funções abaixo servem para estabelecer a conexão com o servidor do PS1 e extrair os dados com a $\textbf{query}$ desejada.

$\textbf{parameters:}$ 
- input: recebe o $\mathit{nside}$ (a resolução) e o $\mathit{pixel}$ que será extraído os dados;
- output: devolve os angulos do centro do pixel e o raio do centro ao vértice mais distante deste.

$\textbf{query\_strings:}$ 
- input: recebe os ângulos do centrais, $\mathit{ang0}$ ($\theta = dec$) e $\mathit{ang1}$ ($\phi =ra$), e o raio de coleta de dados;
- output: a $\mathit{query}$ para comunicar com o servidor.

$\textbf{query\_function:}$ 
- input: $\mathit{params}$ que contêm as informações gerais da coleta de dados e $\mathit{constraints}$ que define as restrições;
- output: a tabela já sob constraints.

Ele estabelecerá a conexão com o PS1 e tentará extrair os dados brutos na região do céu definida. Caso a quantidade de dados seja muito alta, será tratado em uma outra parte do código. Caso não, ao extrair os dados este são estabelecidos numa tabela ascii e selecionado somente os objetos que satisfazem a constraints.

$\textbf{query_constraints:}$ 
- input: $\mathit{table}$ com os dados do PS1 e $\mathit{constraints}$ que define as restrições;
- output: tabela com os dados que satisfazem o constraints.

Ele receberá se haverá separação de galaxias ou de estrelas, ou nenhum dos dois. No caso de ter constraints mas não ter especificação do tipo de objeto, só haverá restrição dos valores dos angulos e das magnitudes (na dada banda) serem >-999. No caso, de não usar restrição, para que seja possível armazenar os dados nos padrões do código, só serão aplicadas as restrições angulares.

In [23]:
def fixcolnames(tab):
    """Fix column names returned by the casjobs query
    
    Parameters
    ----------
    tab (astropy.table.Table): Input table

    Returns reference to original table with column names modified"""

    pat = re.compile(r'\[(?P<name>[^[]+)\]')
    for c in tab.colnames:
        m = pat.match(c)
        if not m:
            raise ValueError("Unable to parse column name '{}'".format(c))
        newname = m.group('name')
        tab.rename_column(c,newname)
    return tab

In [24]:
def parameters(nside,pixel):
    radius = hp.pixelfunc.max_pixrad(nside, degrees=True)*3600
    angles = hp.pix2ang(nside,int(pixel), nest = False, lonlat=True)
    return angles, radius
    

def query_string(ang0,ang1,radius):
        query = """select sot.objID, sot.uniquePspsSTid, sot.ippObjID, sot.surveyID, sot.tessID, sot.projectionID, sot.skyCellID, sot.randomStackObjID, sot.primaryDetection, sot.bestDetection, sot.dvoRegionID, sot.processingVersion,
 sot.gippDetectID, sot.gstackDetectID, sot.gstackImageId, sot.gra, sot.gdec, sot.graErr, sot.gdecErr, sot.gEpoch, sot.gPSFMag, sot.gPSFMagErr, sot.gApMag, sot.gApMagErr, sot.gKronMag, sot.gKronMagErr, sot.ginfoFlag, sot.ginfoFlag2, sot.ginfoFlag3, sot.gnFrames,
 sot.rippDetectID, sot.rstackDetectID, sot.rstackImageId, sot.rra, sot.rdec, sot.rraErr, sot.rdecErr, sot.rEpoch, sot.rPSFMag, sot.rPSFMagErr, sot.rApMag, sot.rApMagErr, sot.rKronMag, sot.rKronMagErr, sot.rinfoFlag, sot.rinfoFlag2, sot.rinfoFlag3, sot.rnFrames,
 sot.iippDetectID, sot.istackDetectID, sot.istackImageId, sot.ira, sot.idec, sot.iraErr, sot.idecErr, sot.iEpoch, sot.iPSFMag, sot.iPSFMagErr, sot.iApMag, sot.iApMagErr, sot.iKronMag, sot.iKronMagErr, sot.iinfoFlag, sot.iinfoFlag2, sot.iinfoFlag3, sot.inFrames,
 sot.zippDetectID, sot.zstackDetectID, sot.zstackImageId, sot.zra, sot.zdec, sot.zraErr, sot.zdecErr, sot.zEpoch, sot.zPSFMag, sot.zPSFMagErr, sot.zApMag, sot.zApMagErr, sot.zKronMag, sot.zKronMagErr, sot.zinfoFlag, sot.zinfoFlag2, sot.zinfoFlag3, sot.znFrames,
 sot.yippDetectID, sot.ystackDetectID, sot.ystackImageId, sot.yra, sot.ydec, sot.yraErr, sot.ydecErr, sot.yEpoch, sot.yPSFMag, sot.yPSFMagErr, sot.yApMag, sot.yApMagErr, sot.yKronMag, sot.yKronMagErr, sot.yinfoFlag, sot.yinfoFlag2, sot.yinfoFlag3, sot.ynFrames
 

 from fGetNearbyObjEq("""+",".join([str(ang0),str(ang1),str(radius/60.)])+""") nb
 inner join StackObjectThin sot on sot.objid=nb.objid

 where sot.primaryDetection = 1 
""" 
        return query

def query_function(params, constraints):
    params['ang'],params['r'] = parameters(params["NSIDE"],params['pixel'])    
    query   =  query_string(params['ang'][0],params['ang'][1],params['r'])
    jobs    = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
    
    try:
        table = jobs.quick(query, task_name="python cone search")
    except Exception:
        print("Exception. code!=200")
        table =  handling_exception(params,constraints)
        print("Extracted {} objects from PS1".format(len(table)))
        return table, jobs  

    table = fixcolnames(ascii.read(table))
    table = query_constraints(table, constraints)
    return table, jobs

def query_constraints(table,constraints):
    band_KronMag = table[''.join([constraints["band"],'KronMag'])]
    band_PSFMag  = table[''.join([constraints["band"],'PSFMag'])]

    if constraints['use']:
        if constraints["type"]=="galaxy":
            constraint = (band_KronMag - band_PSFMag) + 0.192 - 0.120*(band_KronMag - 21.) - 0.018*(band_KronMag - 21.)*(band_KronMag - 21.)
            list1 = np.where((table['gdec']>-999)*(table['gra']>-999)*(band_KronMag>-999)*(band_PSFMag>-999)*(constraint>0))  
        
        elif constraints["type"]=="star":
            constraint = (band_KronMag - band_PSFMag) + 0.192 - 0.120*(band_KronMag - 21.) - 0.018*(band_KronMag - 21.)*(band_KronMag - 21.)
            list1 = np.where((table['gdec']>-999)*(table['gra']>-999)*(band_KronMag>-999)*(band_PSFMag>-999)*(constraint<0))
        
        else:    
            list1 = np.where((table['gdec']>-999)*(table['gra']>-999)*(band_KronMag>-999)*(band_PSFMag>-999))
    else:
        list1 = np.where((table['gdec']>-999)*(table['gra']>-999))
    
    return table[list1]

## Abaixo temos duas funções.

$\textbf{file\_verification:}$ Ele verificará se há arquivo com o mesmo nome do arquivo ".fits" a ser gerado, no diretório FITS (caso não exista, ele criará este). Se tiver o arquivo, ele apaga para poder substituir. Caso não exista ele não faz nada.

$\textbf{galaxies\_pixel:}$ Como coletamos dados a partir de um raio r, devemos tomar um r tal que englobe todo pixel no momento de coletar os dados. Coletado, devemos verificar quais galáxias tem angulos que realmente se encontram no pixel desejado, os que não estiverem serão descartados.

Se as funções do healpy estiverem com $\textbf{lonlat}=True$ então as entradas (ou saídas) serão dadas (longitude,latitude), se for $False$, (colatitude,longitude).

$False$ 

$\textbf{colatitude:}\ \theta \in (0,\pi)$, com $0$ no polo norte.

$\textbf{longitude:}\ \phi \in (0,2\pi)$, com $0$ no polo norte. O $0$ está no centro do mapa e o ângulo aumenta para a esquerda (oeste) do mapa.

$True$ 

$\textbf{longitude:}\ \theta \in (0,2\pi)$, com $0$ no polo norte. O $0$ está no centro do mapa e o ângulo aumenta para a esquerda (oeste) do mapa.

$\textbf{latitude:}\ \phi \in (-\pi/2,\pi/2)$, com $\pi/2$ no polo norte.

$\textbf{name2pixel:}$ Recebe uma string com o nome do arquivo e só devolve a parte que contem o pixel, como um inteiro.

$\textbf{lastpix:}$ Recebe o NSIDE que está sendo rodado o código e verifica se há fits para este nside, caso sim, fornece o último pixel que há no diretório. O programa rodará a partir deste último pixel pois ele pode estar corrompido.

In [25]:
def file_verification(path,file,nside):
    if os.path.isdir(path):#verifica se existe /FITS
        path = os.path.join(path,str(nside)) #.../FITS --> .../FIT/NPIX
        if os.path.isdir(path): # verifica se existe /NPIX
            path = os.path.join(path,file) #nome da path do arquivo
            if os.path.isfile(path): #se existir arquivo com o nome é apagado
                os.remove(path)
        else:
            os.mkdir(path)
    else:
        os.mkdir(path)
        os.mkdir(os.path.join(path,str(nside)))

def galaxies_pixel(table,params):
    list1 = np.where(hp.ang2pix(params['NSIDE'],table['gra'],table['gdec'], lonlat=True) == params['pixel'])
    tab   = table[list1]
    print("Number of galaxies: {}".format(len(tab)))
    return  tab

def name2pixel(name):
	pix = int(name.split("_")[-1].split(".")[0])
	return pix

def lastpix(nside, wpix="last"): #wpix = what pixel?
	path      = os.getcwd()
	pathfits  = os.path.join(path,"FITS",str(nside))
	try:
		files     = os.listdir(pathfits)
		paths     = [os.path.join(pathfits,name) for name in files]
		nfits     = len(paths)
		
		if   wpix == "last":
			lastfits  = max(paths, key=os.path.getctime)
			print("There are {} fits files in : {}".format(nfits,pathfits))
			last      = name2pixel(lastfits)
			print("The last pix in directory is: {}".format(last))
			return last
		elif wpix == "first":
			firstfits = min(paths, key=os.path.getctime)
			print("There are {} fits files in : {}".format(nfits,pathfits))
			first     = name2pixel(firstfits)
			print("The first pix in directory is: {}".format(first)) 
			return first
		else:
			raise Exception("Invalid command.")
	except:
		print("There aren't files in: {}".format(pathfits))
		return False

## FLAGS

Transforma o Hexadecimal usado em binário e verifica quais flags (de uma das três tabelas) são ativadas, e quais dados satisfazem-nas.

$\textbf{hexa2bin:}$ transforma um hexadecimal em binário.

$\textbf{flags\_verification:}$ 
- input: $\mathit{params\_flags}$ que contêm as informações gerais das flags como se será usada, qual das tabelas flags e em que banda. $\mathit{params_hexa}$ fornece o tamanho do hexadecimal (em binario) e ele como binario;
- output: as flags da tabela que são ativadas.

$\textbf{hexa2flags:}$ 
- input: $\mathit{hexa}$ hexadecimal de restrição, $\mathit{file\_names}$ o nome do arquivo com os nomes das flags e $\mathit{file\_datas}$ com o nome do arquivo com os valores das flags;
- output: retorna os dados que não tem os flags ativados..


recebe um (hexa)decimal e devolve as flags ativadas, duma certa tabela de flags.

$\textbf{accept\_data:}$ 
- input: $\mathit{flags\_table}$ as flags a serem verificadas e $\mathit{flags\_input}$ as flags do objeto;
- output: se não há flag ativada pelo objeto, das flags selecionadas pela hexadecimal inicial, o objeto é aceito, caso contrário descartado.


$\textbf{flags\_constraints:}$ 
- input: $\mathit{tab}$ tabela com os dados dos objetos, $\mathit{hexa\_query}$ com a hexadecimal exigida e $\mathit{params}$ com dados gerais da extração;
- output: tabela somente com os dados que não ativam nenhuma flag.

In [26]:
def hex2bin(hexa):
    return bin(int(hexa))[2:]

def flags_verification(params_flags, params_hexa): #verifica quais as flags ativadas
    flags = []
    for i in range(params_flags['num flags']):
        bin_flagi = hex2bin(params_flags['datas'][i])
        len_flagi = len(bin_flagi)
        
        for j in range(len_flagi):
            bin_q = params_hexa['binary hexa'][-j-1]
            bin_f = bin_flagi[-j-1]
            if bin_f=="1" and bin_q=="1":
                flags = np.append(flags,params_flags['names'][i][:-1])
                #print("\nTable       : Detection{}\nFLAG ativada: {}POS_FLAG    : {}".format(params_flags['file'][:-4],params_flags['names'][i],i))
                break
        if len_flagi+1>params_hexa['length hexa']: return flags#break
        
def hexa2flags(hexa,file_names,file_datas): #recebe um (hexa)decimal e devolve as flags ativadas, duma certa tabela de flags
    bin_query = hex2bin(hexa)
    len_query = len(bin_query)
    
    file_names = os.path.join("FLAGS",file_names)
    file_datas = os.path.join("FLAGS",file_datas)
    
    file_     = open(file_names,'r')
    names     = file_.readlines()
    file_.close()
    datas     = np.loadtxt(file_datas)
    num_flags = len(datas)
    
    params_flags = {"names":names, "file":file_names,"datas":datas,"num flags":num_flags}
    params_hexa  = {"length hexa":len_query, "binary hexa":bin_query}
    
    return flags_verification(params_flags,params_hexa)

def accept_data(flags_table, flags_input):
    if set(flags_input).intersection(set(flags_table))==set():
        return True
    else:
        return False

def flags_constraints(tab,hexa_query,params):
    
    file_names = "".join(("FLAGS",str(params["table"]),".txt"))
    file_datas = "".join(("DATA_FLAGS",str(params["table"]),".txt"))
    
    params["bands"] = ["g","r","i","z","y"]
    lbands          = len(params["bands"])
    flags_query     = hexa2flags(hexa_query,file_names,file_datas)
    
    if params["table"]: flag_tab = tab["".join((params["bands"][0],"infoFlag"))]
    else: flag_tab = tab["".join((params["bands"][0],"infoFlag",str(params["table"])))]
    pos = []
    
    for j in range(len(flag_tab)):
        flags_tab    = hexa2flags(flag_tab[j],file_names,file_datas)
        if accept_data(flags_tab,flags_query): pos = np.append(pos,j)
        else: pass
    
    if lbands-1:           
        for i in range(lbands-1): 
            i+=1
            if params["table"]: flag_tab = tab["".join((params["bands"][i],"infoFlag"))]
            else: flag_tab = tab["".join((params["bands"][i],"infoFlag",str(params["table"])))]
            pos2 = []
            for j in range(len(flag_tab)):
                flags_tab    = hexa2flags(flag_tab[j],file_names,file_datas)
                if accept_data(flags_tab,flags_query):
                    pos2 = np.append(pos2,j)
                else: pass
            pos = set(pos).intersection(set(pos2))
    pos = map(int,list(pos))
    return  tab[pos]

## Strips
Selecionar os pixels dentro de um strip


$\textbf{range\_dec:}$ 

$\mathit{params}$ com as informção se será utilizada a restrição em dec e se sim, dá o intervalo desta restrição;


$\textbf{pixelstrips:}$ 
- input: $\mathit{params}$ com as informção se será utilizada a restrição em dec e se sim, dá o intervalo desta restrição;
- output: os pixels a serem extraido dados, que estão dentro da strip.

$\textbf{col2dec:}$ converte colatitude em declination (deg_out=True a saida será em graus, caso contrario em radianos)

$\textbf{dec2col:}$ converte declination em colatitude (deg_out=True a saida será em graus, caso contrario em radianos)

In [27]:
def range_dec(params):
    if params['dec strips']:
        return params['dec center'] - params['dec width']/2.,params['dec center'] + params['dec width']/2.
    else:
        return [-90.,90.]
    
def pixelstrips(params):
    dec1,dec0 = range_dec(params)
    
    nside     = params['NSIDE']
    theta_min = np.array(dec2col(dec1))
    theta_max = np.array(dec2col(dec0))
    pix = []
    if theta_min.size==1:
        pix = np.append(pix,hp.query_strip(nside,theta_max,theta_min, inclusive=True))
    else:
        for i in range(theta_min.size):
            pix = np.append(pix,hp.query_strip(nside,theta_max[i],theta_min[i], inclusive=True))
    return pix.astype(int)

def col2dec(colat=None, deg_out=False):
    colat = np.array(colat)
    ones  = np.ones(colat.size)
    
    down = np.where(colat<0)[0]
    up   = np.where(colat>180)[0]
    
    if (down.size+up.size)==0:
        if deg_out:
            return -(colat - 90.)
        else:
            return np.radians(-(colat - 90.))
    else: print("Error")
        
def dec2col(dec=None, deg_out=False):
    dec  = np.array(dec)
    ones = np.ones(dec.size)
    
    down = np.where(dec<-90)[0]
    up   = np.where(dec>90)[0]
    
    if (down.size+up.size)==0:
        if deg_out:
            return -(dec - 90.)
        else:
            return np.radians(-(dec - 90.))
    else: 
        print("Error")

def newtrips(strips,pix):
	
	wispix = np.where(strips==pix)[0] #wispix = where is pix?
	wispix = int(wispix)
	return strips[wispix:]

# Tratando exceções

Esta parte tratará dos casos em que dá erro de servidor, ou seja, o tamanho dos arquivos são muito grandes. Isto ocorre quando o NSIDE é relativamente baixo, ou seja, a resolução é alta (o tamanho do pixel é grande).
A ideia é verificar de antemão quando vai ocorrer um erro, "quebrar" o pixel em pixels menores e coletar os dados em pixels menores (com raio menor). Após ter os dados, juntar os pixels em um único pixel e verificar quais dados realmente estão no pixel e quais não estão.

Para isto ser efetivo, foi criado uma função $\textbf{maxnside()}$ que identifica qual o NSIDE que dará problema de conexão (devido a conexão utilizada). Com esta informação, crio uma variável $\textbf{NSIDEmax}$ como sendo $4*\textbf{maxnside()}$, que seria o NSIDE que a conexão certamente não dará problema. O fator é $2^2 =4$ pois, para $2$ pode ser que exista pixels que deem problema, mas não deram pois a verificação é feita na região central $(\theta,\phi)=(0,0)$.

Assim, quando o código acusar $\textbf{Exception}$ no dado pixel, ou seja, quando a quantidade de dados a serem extraidos do servidor for muito alto, o pixel é "quebrado" em pixels menores ($\textbf{NSIDEmax}$) que cobrem o pixel e os dados são extraidos em pixels menores. Ao final da extração, faz-se uma verificação quais dados de fato pertencem ao pixel original.

In [28]:
def handling_exception(params,constraints):
    NSIDEmax = params['NSIDE max']
    vec      = hp.ang2vec(params["ang"][0],params["ang"][1], lonlat=True)
    pixels   = hp.query_disc(NSIDEmax,vec,radians(params['r']/3600.), inclusive= True)
    subjobs  = mastcasjobs.MastCasJobs(context="PanSTARRS_DR2")
    
    for pixel in pixels:
        ang,r     = parameters(NSIDEmax,pixel)
        subquery  = sub_query_string(ang[0],ang[1],r)
        accept    = True
        
        while accept:
            try:
                subtab = subjobs.quick(subquery, task_name="python cone search")
                accept = False
            except Exception:
                time.sleep(60)
                pass
   
        subtab    = fixcolnames(ascii.read(subtab))
        subtab    = query_constraints(subtab, constraints)
        
        if pixel == pixels[0]:
            table = subtab
        else:
            table = vstack([table,subtab])

    return table

def sub_query_string(ang0,ang1,r):
        query = """select sot.objID, sot.uniquePspsSTid, sot.ippObjID, sot.surveyID, sot.tessID, sot.projectionID, sot.skyCellID, sot.randomStackObjID, sot.primaryDetection, sot.bestDetection, sot.dvoRegionID, sot.processingVersion,
 sot.gippDetectID, sot.gstackDetectID, sot.gstackImageId, sot.gra, sot.gdec, sot.graErr, sot.gdecErr, sot.gEpoch, sot.gPSFMag, sot.gPSFMagErr, sot.gApMag, sot.gApMagErr, sot.gKronMag, sot.gKronMagErr, sot.ginfoFlag, sot.ginfoFlag2, sot.ginfoFlag3, sot.gnFrames,
 sot.rippDetectID, sot.rstackDetectID, sot.rstackImageId, sot.rra, sot.rdec, sot.rraErr, sot.rdecErr, sot.rEpoch, sot.rPSFMag, sot.rPSFMagErr, sot.rApMag, sot.rApMagErr, sot.rKronMag, sot.rKronMagErr, sot.rinfoFlag, sot.rinfoFlag2, sot.rinfoFlag3, sot.rnFrames,
 sot.iippDetectID, sot.istackDetectID, sot.istackImageId, sot.ira, sot.idec, sot.iraErr, sot.idecErr, sot.iEpoch, sot.iPSFMag, sot.iPSFMagErr, sot.iApMag, sot.iApMagErr, sot.iKronMag, sot.iKronMagErr, sot.iinfoFlag, sot.iinfoFlag2, sot.iinfoFlag3, sot.inFrames,
 sot.zippDetectID, sot.zstackDetectID, sot.zstackImageId, sot.zra, sot.zdec, sot.zraErr, sot.zdecErr, sot.zEpoch, sot.zPSFMag, sot.zPSFMagErr, sot.zApMag, sot.zApMagErr, sot.zKronMag, sot.zKronMagErr, sot.zinfoFlag, sot.zinfoFlag2, sot.zinfoFlag3, sot.znFrames,
 sot.yippDetectID, sot.ystackDetectID, sot.ystackImageId, sot.yra, sot.ydec, sot.yraErr, sot.ydecErr, sot.yEpoch, sot.yPSFMag, sot.yPSFMagErr, sot.yApMag, sot.yApMagErr, sot.yKronMag, sot.yKronMagErr, sot.yinfoFlag, sot.yinfoFlag2, sot.yinfoFlag3, sot.ynFrames
 

 from fGetNearbyObjEq("""+",".join([str(ang0),str(ang1),str(r/60.)])+""") nb
 inner join StackObjectThin sot on sot.objid=nb.objid

 where sot.primaryDetection = 1 
""" 

        return query


## Escrever e ler arquivo ".fits" dos dados óticos

In [29]:
def write_fits(table,params):
    file = "_".join(("PixelFit",str(params['NSIDE']),str(int(params['pixel']))))
    file = ".".join((file,"fits"))
    path = os.getcwd()
    path = os.path.join(path,"FITS")
    file_verification(path,file,params['NSIDE'])
    path = os.path.join(path,str(params['NSIDE']))
    table.write(os.path.join(path,file))

    
def read_fit(params): 
    #params = {"NSIDE":NSIDE, 'pixel':pix, "NPIX":NPIX}
    params['ang'] = hp.pix2ang(params['NSIDE'],int(params['pixel']), nest = True, lonlat=True)
    file_name = "_".join(["PixelFit",str(params['NSIDE']),str(params['pixel'])])
    file_name = ".".join([file_name,"fits"])
    
    tab = Table.read(os.path.join("FITS",str(params['NSIDE']),file_name))

    return tab, params

# O programa começa a partir daqui

In [30]:
NPIX   = hp.nside2npix(NSIDE)
params = {"NSIDE":NSIDE, "NPIX":NPIX} 
params["NSIDE max"] = 8*maxnside()
params_strips['NSIDE']=params['NSIDE']
print( "NSIDE      : {}".format(params['NSIDE']))
print( "Num. Pixels: {}".format(params['NPIX']))
print( "NSIDE   max: {}".format(params['NSIDE max']))

NSIDE      : 1024
Num. Pixels: 12582912
NSIDE   max: 1024


In [42]:
strips = pixelstrips(params_strips) if params_strips['dec strips'] else np.arange(params['NPIX'])
if not restart:
	last   = lastpix(NSIDE,"last")
	if last:
		strips = newtrips(strips,last)
	else: pass	
print("{:.2f}% of the sky covered.".format(100*float(len_strips)/params['NPIX']))

There are 8 fits files in : /home/marins/Documentos/Programmation/PS1zxcorr/notebook/FITS/1024
The last pix in directory is: 7108615
12.66% of the sky covered.


In [ ]:

for num,pix in enumerate(strips):    
    timei     = time()
    theta,phi = hp.pix2ang(params['NSIDE'],pix, lonlat=True, nest=False)
    
    params['pixel'] = pix
    tab, job        = query_function(params, constraints)
    tab             = galaxies_pixel(tab,params)
    tab             = flags_constraints(tab,hexa_query,params_flags)
    write_fits(tab,params)
    
    
    timef    = strftime('%H:%M:%S', gmtime(time()-timei))
    print("Program's time (hh:mm:ss): {}".format(timef))
    print("Pixel {}".format(pix))
    print("{:.2f}% completed program\n \n".format(100*(float(num+1)/len_strips)))


Number of galaxies: 125
Program's time (hh:mm:ss): 00:00:02
Pixel 7108615
0.00% completed program
 

Number of galaxies: 140
Program's time (hh:mm:ss): 00:00:02
Pixel 7108616
0.00% completed program
 

